https://github.com/JuliaPOMDP/POMDPs.jl

In [1]:
using POMDPs
using Pkg

In [2]:
POMDPs.add_registry()

   Cloning registry from "https://github.com/JuliaPOMDP/Registry"
     Added registry `JuliaPOMDP` to `~/.julia/registries`0 %========>                                ]  18.3 %22.2 %>                       ]  40.5 % %>              ]  64.4 % [==============================>          ]  73.9 %   ]  92.0 %Fetching: [=======================================> ]  96.8 %


In [3]:
Pkg.add.(["QuickPOMDPs", "POMDPSimulators", "QMDP"])

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 %Fetching: [=======================================> ]  96.7 %  Updating registry at `~/.julia/registries/JuliaPOMDP`
  Updating git-repo `https://github.com/JuliaPOMDP/Registry`
 Resolving package versions...
 Installed QuickPOMDPs ───── v0.1.0
 Installed POMDPTesting ──── v0.1.0
 Installed BeliefUpdaters ── v0.1.2
 Installed POMDPModelTools ─ v0.1.6
  Updating `~/.julia/environments/v1.1/Project.toml`
  [8af83fb2] + QuickPOMDPs v0.1.0
  Updating `~/.julia/environments/v1.1/Manifest.toml`
  [8bb6e9a1] + BeliefUpdaters v0.1.2
  [08074719] + POMDPModelTools v0.1.6
  [92e6a534] + POMDPTesting v0.1.0
  [8af83fb2] + QuickPOMDPs v0.1.0
 Resolving package versions...
 Installed TableTraits ───────────────── v1.0.0
 Installed InvertedIndices ───────────── v1.0.0
 Installed DataValueInterfaces ───────── v1.0.0

┌ Info: Installed QMDP ─────────────────── v0.1.1
└ @ Pkg.Operations /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.1/Pkg/src/Operations.jl:637


3-element Array{Nothing,1}:
 nothing
 nothing
 nothing

In [4]:
using POMDPs, QuickPOMDPs, POMDPSimulators, QMDP

┌ Info: Precompiling QuickPOMDPs [8af83fb2-a731-493c-9049-9e19dbce6165]
└ @ Base loading.jl:1186
┌ Info: Precompiling POMDPSimulators [e0d0a172-29c6-5d4e-96d0-f262df5d01fd]
└ @ Base loading.jl:1186
┌ Info: Precompiling QMDP [3aa3ecc9-5a5d-57c8-8188-3e47bd8068d2]
└ @ Base loading.jl:1186


In [5]:
S = [:left, :right]           # S, A, and O may contain any objects
A = [:left, :right, :listen]  # including user-defined types
O = [:left, :right]
γ = 0.95

function T(s, a, sp)
    if a == :listen
        return s == sp
    else # a door is opened
        return 0.5 #reset
    end
end

function Z(a, sp, o)
    if a == :listen
        if o == sp
            return 0.85
        else
            return 0.15
        end
    else
        return 0.5
    end
end

function R(s, a)
    if a == :listen  
        return -1.0
    elseif s == a # the tiger was found
        return -100.0
    else # the tiger was escaped
        return 10.0
    end
end

m = DiscreteExplicitPOMDP(S,A,O,T,Z,R,γ)

DiscreteExplicitPOMDP{Symbol,Symbol,Symbol,typeof(Z),typeof(R)}(Symbol[:left, :right], Symbol[:left, :right, :listen], Symbol[:left, :right], Dict((:left, :right)=>SparseCat{Array{Symbol,1},Array{Float64,1}}(Symbol[:left, :right], [0.5, 0.5]),(:left, :listen)=>SparseCat{Array{Symbol,1},Array{Float64,1}}(Symbol[:left], [1.0]),(:right, :left)=>SparseCat{Array{Symbol,1},Array{Float64,1}}(Symbol[:left, :right], [0.5, 0.5]),(:right, :right)=>SparseCat{Array{Symbol,1},Array{Float64,1}}(Symbol[:left, :right], [0.5, 0.5]),(:right, :listen)=>SparseCat{Array{Symbol,1},Array{Float64,1}}(Symbol[:right], [1.0]),(:left, :left)=>SparseCat{Array{Symbol,1},Array{Float64,1}}(Symbol[:left, :right], [0.5, 0.5])), Dict((:listen, :right)=>SparseCat{Array{Symbol,1},Array{Float64,1}}(Symbol[:left, :right], [0.15, 0.85]),(:left, :right)=>SparseCat{Array{Symbol,1},Array{Float64,1}}(Symbol[:left, :right], [0.5, 0.5]),(:right, :left)=>SparseCat{Array{Symbol,1},Array{Float64,1}}(Symbol[:left, :right], [0.5, 0.5]),

In [6]:
solver = QMDPSolver()
policy = solve(solver, m)

POMDPPolicies.AlphaVectorPolicy{DiscreteExplicitPOMDP{Symbol,Symbol,Symbol,typeof(Z),typeof(R)},Symbol}(DiscreteExplicitPOMDP{Symbol,Symbol,Symbol,typeof(Z),typeof(R)}(Symbol[:left, :right], Symbol[:left, :right, :listen], Symbol[:left, :right], Dict((:left, :right)=>SparseCat{Array{Symbol,1},Array{Float64,1}}(Symbol[:left, :right], [0.5, 0.5]),(:left, :listen)=>SparseCat{Array{Symbol,1},Array{Float64,1}}(Symbol[:left], [1.0]),(:right, :left)=>SparseCat{Array{Symbol,1},Array{Float64,1}}(Symbol[:left, :right], [0.5, 0.5]),(:right, :right)=>SparseCat{Array{Symbol,1},Array{Float64,1}}(Symbol[:left, :right], [0.5, 0.5]),(:right, :listen)=>SparseCat{Array{Symbol,1},Array{Float64,1}}(Symbol[:right], [1.0]),(:left, :left)=>SparseCat{Array{Symbol,1},Array{Float64,1}}(Symbol[:left, :right], [0.5, 0.5])), Dict((:listen, :right)=>SparseCat{Array{Symbol,1},Array{Float64,1}}(Symbol[:left, :right], [0.15, 0.85]),(:left, :right)=>SparseCat{Array{Symbol,1},Array{Float64,1}}(Symbol[:left, :right], [0.5

In [7]:
rsum = 0.0
for (s,b,a,o,r) in stepthrough(m, policy, "s,b,a,o,r", max_steps=10)
    println("s: $s, b: $([pdf(b,s) for s in S]), a: $a, o: $o")
    global rsum += r
end
println("Undiscounted reward was $rsum.")

s: left, b: [0.5, 0.5], a: listen, o: left
s: left, b: [0.85, 0.15], a: listen, o: left
s: left, b: [0.969799, 0.0302013], a: right, o: left
s: left, b: [0.5, 0.5], a: listen, o: left
s: left, b: [0.85, 0.15], a: listen, o: left
s: left, b: [0.969799, 0.0302013], a: right, o: left
s: right, b: [0.5, 0.5], a: listen, o: right
s: right, b: [0.15, 0.85], a: listen, o: right
s: right, b: [0.0302013, 0.969799], a: left, o: right
s: left, b: [0.5, 0.5], a: listen, o: right
Undiscounted reward was 23.0.


In [9]:
using Flux

In [11]:
Pkg.status("Flux")

    Status `~/.julia/environments/v1.1/Project.toml`
  [587475ba] Flux v0.8.3


In [15]:
Pkg.clone("DeepQLearning")
Pkg.build("DeepQLearning")

  Building Arpack ──────────→ `~/.julia/packages/Arpack/cu5By/deps/build.log`
  Building SpecialFunctions → `~/.julia/packages/SpecialFunctions/fvheQ/deps/build.log`
  Building Rmath ───────────→ `~/.julia/packages/Rmath/Py9gH/deps/build.log`
  Building ZMQ ─────────────→ `~/.julia/packages/ZMQ/ABGOx/deps/build.log`
  Building ZipFile ─────────→ `~/.julia/packages/ZipFile/oD4uG/deps/build.log`
  Building CodecZlib ───────→ `~/.julia/packages/CodecZlib/9jDi1/deps/build.log`
  Building Conda ───────────→ `~/.julia/packages/Conda/kLXeC/deps/build.log`
  Building FFTW ────────────→ `~/.julia/packages/FFTW/2okGQ/deps/build.log`


In [16]:
using DeepQLearning

┌ Info: Precompiling DeepQLearning [de0a67f4-c691-11e8-0034-5fc6e16e22d3]
└ @ Base loading.jl:1186


In [22]:
Pkg.add(["POMDPSimulators", "POMDPModels"])

 Resolving package versions...
 Installed Measures ───── v0.3.0
 Installed IterTools ──── v1.2.0
 Installed Compose ────── v0.7.3
 Installed POMDPModels ── v0.3.5
 Installed ColorSchemes ─ v3.3.0
  Updating `~/.julia/environments/v1.1/Project.toml`
  [355abbd5] + POMDPModels v0.3.5
  Updating `~/.julia/environments/v1.1/Manifest.toml`
  [35d6a980] + ColorSchemes v3.3.0
  [a81c6b42] + Compose v0.7.3
  [c8e1da08] + IterTools v1.2.0
  [442fdcdd] + Measures v0.3.0
  [355abbd5] + POMDPModels v0.3.5


In [23]:
using DeepQLearning
using POMDPs
using Flux
using POMDPModels
using POMDPSimulators

┌ Info: Precompiling POMDPModels [355abbd5-f08e-5560-ac9e-8b5f2592a0ca]
└ @ Base loading.jl:1186


In [24]:
# load MDP model from POMDPModels or define your own!
mdp = SimpleGridWorld();

In [25]:
# Define the Q network (see Flux.jl documentation)
# the gridworld state is represented by a 2 dimensional vector.
model = Chain(Dense(2, 32), Dense(32, n_actions(mdp)))

Chain(Dense(2, 32), Dense(32, 4))

In [26]:
solver = DeepQLearningSolver(qnetwork = model, max_steps=10000, 
                             learning_rate=0.005,log_freq=500,
                             recurrence=false,double_q=true, dueling=true, prioritized_replay=true)

DeepQLearningSolver
  qnetwork: Chain{Tuple{Dense{typeof(identity),TrackedArray{…,Array{Float32,2}},TrackedArray{…,Array{Float32,1}}},Dense{typeof(identity),TrackedArray{…,Array{Float32,2}},TrackedArray{…,Array{Float32,1}}}}}
  learning_rate: Float64 0.005
  max_steps: Int64 10000
  batch_size: Int64 32
  train_freq: Int64 4
  eval_freq: Int64 500
  target_update_freq: Int64 500
  num_ep_eval: Int64 100
  double_q: Bool true
  dueling: Bool true
  recurrence: Bool false
  eps_fraction: Float64 0.5
  eps_end: Float64 0.01
  evaluation_policy: DeepQLearning.basic_evaluation (function of type typeof(DeepQLearning.basic_evaluation))
  exploration_policy: getfield(DeepQLearning, Symbol("#action_epsilon_greedy#11")){Int64,Float64,Float64}(10000, 0.5, 0.01) (function of type getfield(DeepQLearning, Symbol("#action_epsilon_greedy#11")){Int64,Float64,Float64})
  trace_length: Int64 40
  prioritized_replay: Bool true
  prioritized_replay_alpha: Float64 0.6
  prioritized_replay_epsilon: Float64 1

In [27]:
policy = solve(solver, mdp)

  500 / 10000 eps 0.901 |  avgR -0.083 | Loss 2.005e+00 | Grad 2.324e-01 | EvalR -Inf 
Evaluation ... Avg Reward 1.14 | Avg Step 78.11 
 1000 / 10000 eps 0.802 |  avgR 0.412 | Loss 1.306e+00 | Grad 1.005e-01 | EvalR 1.140 
Evaluation ... Avg Reward -4.47 | Avg Step 51.14 
 1500 / 10000 eps 0.703 |  avgR 0.000 | Loss 1.206e+00 | Grad 2.229e-01 | EvalR -4.470 
Evaluation ... Avg Reward 2.20 | Avg Step 71.12 
 2000 / 10000 eps 0.604 |  avgR 0.473 | Loss 6.163e-01 | Grad 1.213e-01 | EvalR 2.200 
Evaluation ... Avg Reward -1.44 | Avg Step 48.51 
 2500 / 10000 eps 0.505 |  avgR 0.183 | Loss 5.707e-01 | Grad 7.312e-02 | EvalR -1.440 
Evaluation ... Avg Reward 3.54 | Avg Step 48.56 
 3000 / 10000 eps 0.406 |  avgR -0.100 | Loss 6.952e-01 | Grad 2.380e-01 | EvalR 3.540 
Evaluation ... Avg Reward 1.20 | Avg Step 60.61 
Saving new model with eval reward 1.200 
 3500 / 10000 eps 0.307 |  avgR -0.522 | Loss 7.535e-01 | Grad 7.019e-02 | EvalR 1.200 
Evaluation ... Avg Reward 2.41 | Avg Step 56.72 
 

NNPolicy{SimpleGridWorld,DeepQLearning.DuelingNetwork,Symbol}(SimpleGridWorld
  size: Tuple{Int64,Int64}
  rewards: Dict{StaticArrays.SArray{Tuple{2},Int64,1,2},Float64}
  terminate_from: Set{StaticArrays.SArray{Tuple{2},Int64,1,2}}
  tprob: Float64 0.7
  discount: Float64 0.95
, DeepQLearning.DuelingNetwork(Chain(), Chain(Dense(2, 32), Dense(32, 1)), Chain(Dense(2, 32), Dense(32, 4))), Symbol[:up, :down, :left, :right], 1)

In [32]:
sim = RolloutSimulator(max_steps=30)
r_tot = simulate(sim, mdp, policy)
println("Total discounted reward for 1 simulation: $r_tot")

Total discounted reward for 1 simulation: 2.7075
